# Langkah-langkah dan Implementasi SVM

**1. Persiapan Dataset**

In [3]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

def load_and_preprocess_dataset(dataset_path, image_size=(28, 28)):
    """
    Fungsi untuk memuat dataset dari folder.
    Args:
        dataset_path (str): Path ke folder dataset.
        image_size (tuple): Ukuran gambar yang akan di-resize.
    Returns:
        X_train, X_test, y_train, y_test: Dataset yang sudah diproses dan dibagi.
    """
    X = []
    y = []

    # Iterasi semua subfolder sebagai label
    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        if os.path.isdir(label_path):  # Pastikan ini folder
            for file_name in os.listdir(label_path):
                file_path = os.path.join(label_path, file_name)
                try:
                    # Membaca dan memproses gambar
                    img = Image.open(file_path).convert('L')  # Grayscale
                    img = img.resize(image_size)  # Resize gambar
                    X.append(np.array(img))  # Tambahkan gambar
                    y.append(int(label))  # Tambahkan label
                except Exception as e:
                    print(f"Error membaca file {file_path}: {e}")

    # Konversi ke array numpy
    X = np.array(X) / 255.0  # Normalisasi
    X = X.reshape(X.shape[0], -1)  # Flatten gambar
    y = np.array(y)

    # Membagi dataset
    return train_test_split(X, y, test_size=0.2, random_state=42)

# Path ke folder dataset
dataset_path = "C:/Users/desli/Machine Learning/UAS Machine Learning/Angka"  # Ganti path sesuai dataset Anda

# Load dataset
X_train, X_test, y_train, y_test = load_and_preprocess_dataset(dataset_path)
print(f"Dataset berhasil dimuat!")
print(f"Jumlah data latih: {len(X_train)}, data uji: {len(X_test)}")

Dataset berhasil dimuat!
Jumlah data latih: 1760, data uji: 440


**2. Inisialisasi dan Pelatihan Model SVM**

In [5]:
from sklearn.svm import SVC

# Inisialisasi model SVM
print("Inisialisasi model SVM...")
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')  # Kernel RBF

# Latih model
print("Melatih model SVM...")
svm_model.fit(X_train, y_train)
print("Model selesai dilatih!")

Inisialisasi model SVM...
Melatih model SVM...
Model selesai dilatih!


**3. Evaluasi Model**

In [7]:
from sklearn.metrics import accuracy_score, classification_report

# Prediksi pada data uji
print("Evaluasi model...")
y_pred = svm_model.predict(X_test)

# Hitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi pada data uji: {accuracy:.2f}")

# Laporan klasifikasi
print("Laporan klasifikasi:")
print(classification_report(y_test, y_pred))

Evaluasi model...
Akurasi pada data uji: 0.45
Laporan klasifikasi:
              precision    recall  f1-score   support

           0       0.50      0.43      0.47        46
           1       0.60      0.61      0.61        54
           2       0.46      0.52      0.49        48
           3       0.45      0.44      0.45        34
           4       0.40      0.40      0.40        45
           5       0.44      0.28      0.34        39
           6       0.27      0.24      0.25        38
           7       0.36      0.50      0.42        42
           8       0.65      0.67      0.66        52
           9       0.30      0.31      0.31        42

    accuracy                           0.45       440
   macro avg       0.44      0.44      0.44       440
weighted avg       0.46      0.45      0.45       440



**4. Simpan Model**

In [9]:
import joblib

# Simpan model SVM
model_path = "svm_model.pkl"
joblib.dump(svm_model, model_path)
print(f"Model SVM disimpan di {model_path}")

Model SVM disimpan di svm_model.pkl


In [10]:
import os
print("Current working directory:", os.getcwd())

Current working directory: C:\Users\desli\Machine Learning\UAS Machine Learning


In [11]:
import pickle

with open('SVM.pkl', 'wb') as file:
    pickle.dump(model_path,file)

**5. Prediksi Gambar Baru**

In [13]:
import os
import numpy as np
from PIL import Image
import joblib
from sklearn.svm import SVC

# Fungsi untuk memuat dan memproses gambar sebelum prediksi
def preprocess_image(image_path, image_size=(28, 28)):
    """
    Memuat dan memproses gambar untuk prediksi.
    Args:
        image_path (str): Path ke gambar.
        image_size (tuple): Ukuran gambar yang akan di-resize.
    Returns:
        np.ndarray: Gambar yang sudah diproses dalam bentuk array 1D.
    """
    try:
        # Memuat gambar dan mengubahnya ke grayscale
        img = Image.open(image_path).convert('L')  # Ubah ke grayscale
        img = img.resize(image_size)  # Ubah ukuran menjadi 28x28
        img_array = np.array(img) / 255.0  # Normalisasi nilai piksel ke [0, 1]

        # Debugging: Tampilkan jumlah piksel aktif (bukan nol)
        print(f"Jumlah piksel aktif (bukan nol): {np.sum(img_array > 0)}")

        img_array = img_array.flatten()  # Flatten ke array 1D
        return img_array
    except Exception as e:
        print(f"Error memproses gambar {image_path}: {e}")
        return None

# Fungsi untuk memprediksi kelas gambar
def predict_new_image(image_path, model, image_size=(28, 28)):
    """
    Prediksi kelas untuk gambar baru.
    Args:
        image_path (str): Path ke gambar baru.
        model: Model SVM yang sudah dilatih.
        image_size (tuple): Ukuran gambar yang akan di-resize.
    Returns:
        int: Kelas prediksi atau None jika terjadi error.
    """
    # Preprocessing gambar
    image_data = preprocess_image(image_path, image_size)
    if image_data is None:
        return None

    # Debugging: Tampilkan informasi gambar
    print(f"Data gambar diproses (panjang array): {len(image_data)}")
    print(f"Jumlah nilai piksel (sum): {np.sum(image_data)}")

    try:
        # Prediksi menggunakan model
        prediction = model.predict([image_data])
        return prediction[0]
    except Exception as e:
        print(f"Error selama prediksi: {e}")
        return None

# Path ke model SVM
model_path = "C:/Users/desli/Machine Learning/UAS Machine Learning/svm_model.pkl"

# Memastikan model tersedia atau inisialisasi baru jika tidak ada model yang disimpan
if os.path.exists(model_path):
    try:
        svm_model = joblib.load(model_path)
        if not hasattr(svm_model, "predict"):
            raise ValueError("Model yang dimuat bukan model SVM yang valid.")
        print("Model SVM berhasil dimuat.")
    except Exception as e:
        print(f"Error saat memuat model: {e}")
        svm_model = None
else:
    print(f"Model tidak ditemukan di {model_path}. Inisialisasi model baru.")
    svm_model = SVC(kernel='rbf', C=1, gamma=0.01)

# Debugging: Pastikan model memiliki support vector
if svm_model and hasattr(svm_model, "support_vectors_"):
    print(f"Jumlah support vector: {len(svm_model.support_vectors_)}")

# Contoh prediksi gambar baru
new_image_path = "C:/Users/desli/Machine Learning/UAS Machine Learning/6.055.5.augmented.png"  # Ganti dengan path gambar baru
if os.path.exists(new_image_path) and svm_model:
    predicted_class = predict_new_image(new_image_path, svm_model)
    if predicted_class is not None:
        print(f"Prediksi untuk gambar baru: {predicted_class}")
    else:
        print("Gagal memprediksi gambar.")
else:
    if not os.path.exists(new_image_path):
        print(f"Gambar baru tidak ditemukan di {new_image_path}")
    if svm_model is None:
        print("Model SVM tidak valid, tidak dapat melakukan prediksi.")

Model SVM berhasil dimuat.
Jumlah support vector: 1737
Jumlah piksel aktif (bukan nol): 771
Data gambar diproses (panjang array): 784
Jumlah nilai piksel (sum): 745.0117647058823
Prediksi untuk gambar baru: 6
